In [ ]:
#All the required imports 

import pandas as pd
from glob import glob
from matplotlib import pyplot as plt
from PIL import Image as im
from array import *
import math
import os
import cv2
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os,random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib.colors import ListedColormap

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directorys
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#train and test dir in google drive 
train_dir='../input/drive-retinal-vessel-segmentation-pixelwise/DRIVE_Modified/Modified_dataset_train'
test_dir='../input/drive-retinal-vessel-segmentation-pixelwise/DRIVE_Modified/Modified_dataset_test'

training_data = (glob(train_dir+'/img*.csv'))
testing_data = (glob(test_dir+'/img*.csv'))


  

In [ ]:
print('Number of training imgs= '+str(len(training_data)) +'\t'+'Number of testing imgs= '+str(len(testing_data)))

In [ ]:
#test and train file iterator 
final_training_data = pd.concat((pd.read_csv(file).assign(filename = file) 
                                for file in training_data), ignore_index = True)
final_testing_data = pd.concat((pd.read_csv(file).assign(filename = file) 
                                for file in testing_data), ignore_index = True)

In [ ]:
x_train=final_training_data[['R','G','B']].values
y_train = (final_training_data['truth labels'].values/255)
x_test=final_testing_data[['R','G','B']].values
y_test = (final_testing_data['truth labels'].values/255)

In [ ]:

x_train_scaled = x_train/255
x_test_scaled = x_test/255


In [ ]:
test_file_img_pth='/content/drive/MyDrive/Datasets/DRIVE_Modified/Modified_dataset_test/img1.csv'
train_file_img_pth='/content/drive/MyDrive/Datasets/DRIVE_Modified/Modified_dataset_train/img1.csv'



test_img=pd.read_csv(test_file_img_pth)
print(test_img.shape)
train_img=pd.read_csv(train_file_img_pth)
print(train_img.shape)

test_img_1=test_img[['R','G','B']].values
test_img_1_scaled=test_img_1/255
train_img_1=train_img[['R','G','B']].values
train_img_1_scaled=train_img_1/255

test_img_1_label=(test_img['truth labels'].values/255)

train_img_1_label=(train_img['truth labels'].values)/255



# oversampling the data

In [ ]:
!pip install -U imbalanced-learn

In [ ]:
import imblearn
imblearn.__version__


In [ ]:
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
adasyn = ADASYN()
smote=SMOTE()
x_train_scaled_oversampled, y_train = smote.fit_resample(x_train_scaled, y_train)
x_train_scaled_oversampled_1, y_train_1 = adasyn.fit_resample(train_img_1_scaled, train_img_1_label)

In [ ]:
#oversampling on one img data
from imblearn.over_sampling import ADASYN
adasyn = ADASYN()
x_train_scaled_oversampled_1, y_train_1 = adasyn.fit_resample(train_img_1_scaled, train_img_1_label)

# XGboost Model

## Trained on only 1 training image pixels 


In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(x_train_scaled_oversampled_1, label=y_train_1)
dtest = xgb.DMatrix(test_img_1_scaled, label=test_img_1_label)

In [ ]:
evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [ ]:
param = {'max_depth': 20, 'eta': 0.4, 'objective': 'binary:hinge','eval_metric':['auc','error'],'verbosity':1,'use_rmm':'true','seed':69420,'validate_parameters':'true','min_child_weight':3,'sampling_method':'gradient_based'}


In [ ]:
num_round = 100
bst = xgb.train(param, dtrain, num_round, evallist)

In [ ]:
bst.save_model('0001.model')

In [ ]:
ypred = bst.predict(dtest)

plt.imshow(test_img['truth labels'].to_numpy().reshape((584,565),order='C'),cmap='gray')

In [ ]:
test_prediction = ypred.reshape((584,565),order='C')
plt.imshow(test_prediction,cmap='gray')

In [ ]:
import matplotlib.image as mpimg
test1_img = mpimg.imread('/content/drive/MyDrive/Datasets/DRIVE/test/images/01_test.tif')
plt.imshow(test1_img)

## XGBoost trained on all training images

In [ ]:
param = {'max_depth': 100, 'eta': 0.4, 'objective': 'binary:hinge','eval_metric':['auc','error'],'verbosity':1,'use_rmm':'true','seed':69420,'validate_parameters':'true','min_child_weight':5,'sampling_method':'gradient_based','gamma':10}


In [ ]:
dtrain_full = xgb.DMatrix(x_train_scaled_oversampled, label=y_train)
dtest_full = xgb.DMatrix(x_test_scaled, label=y_test)


In [ ]:
num_round = 16
bst_full_data = xgb.train(param, dtrain_full, num_round, evallist)

In [ ]:
ypred = bst_full_data.predict(dtest)

In [ ]:
ypred

In [ ]:
test_prediction = ypred.reshape((584,565),order='C')
plt.imshow(test_prediction,cmap='gray')